# Lecture 2.3: Tabular Data Pt.2


This lecture, we are going to **explore** the top 50 songs of 2019 spotify [dataset](https://www.kaggle.com/leonardopena/top50spotify2019) with [pandas](https://pandas.pydata.org/).

**Learning goals:**
- Calculate column statistics
- Group and aggregate rows
- Investigate how variables relate
- Calculate a new variable
- Answer questions about the dataset

---

## 1. Summary Statistics

In lecture 2.2, we saw a couple of tricks to get insights into large datasets. The `.head()` method on `DataFrame` could get us an overview of the columns and the type of data that they hold. The selection methods `[]`, `.loc[]`, and `.iloc[]` allowed us to focus on specific parts of the dataset. Finally, we showed that the `df.plot.line()` method could be used to visualise numeric columns. 

However, it still feels like we are mostly blind to what kind of data is actually held in the dataframe. What's the most common `Genre`? The minimum value of `Liveness`? What's the range of `Danceability`?  This might be somewhat obvious for well known variables like `Beats.Per.Minute`, but we'll often come across variables that we have no intuition for. Therefore being able to _summarize_ the main statistical attributes of each column is key to _understand_ the dataset better. This, in turn, will allow us to _use_ it more effectively in downstream tasks.

Once again, let's load and inspect our [top 50 spotify songs of 2019](https://www.kaggle.com/leonardopena/top50spotify2019) dataset:

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('top50.csv', encoding = "latin")
df.head()

One column that is still particularly enigmatic to me is `Danceability`. I can guess that songs with high `Danceability` probably have a fast, regular beat, but I have no idea of what's a "good" or "bad" `Danceability` score. In other words, we'd like to know the min/max _range_ of the `Danceability` column to better understand the values.

You can guess that this is a common scenario during data exploration, and thus pandas makes this very easy:

In [ ]:
df['Danceability'].min()

In [ ]:
df['Danceability'].max()

Here, we selected the `Danceability` column using the `[]`, which if you remember from last lecture returns a `Series` object, and we then called the `Series` methods [`.min()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.min.html) and [`.max()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.max.html).

We also could have got _all_ of the columns' minimum values by calling [`.min()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.min.html) directly on the `DataFrame`:

In [ ]:
df.min()

Now that we know that `Danceability` $\in [29, 90]$, we're curious to know what's the most danceable song of 2019! One way to achieve this would be to use the [`.sort_values()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) method we've used before, and then pick the first row using `df.iloc[0]`. However, sorting is slow, and we'd be messing with the order of the rows. We could also use a _boolean mask_ checking rows with elements equal to that max value. Whilst that would work, there is an easier way.

To find the _index_ of min or max values, we can use the [`.idxmin()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.idxmin.html) and [`.idxmax()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.idxmax.html) methods.

In [ ]:
df['Danceability'].idxmax()

This returns the _label_ of the row where the max value occurs. Therefore, we can easily fetch the most danceable song of 2019 by combining the above with `.loc[]`:

In [ ]:
df.loc[df['Danceability'].idxmax()]

Very danceable indeed! 💃

We've now got an idea of what's a good `Danceability` score, and an example of a danceable song. But most songs aren't as groovy as Khalid's tune! How do we know what's a _normal_ amount of `Danceability`? To do this, we can calculate the column _average_. As expected, pandas makes this trivial:

In [ ]:
df['Danceability'].mean()

Wow, it seems popular songs are quite danceable on average! As we know from the minimum value = 29, there must still be some variation. Calculating the _standard deviation_ of the column is useful to gauge this variation:

In [ ]:
df['Danceability'].std()

Calculating ranges, means, and standard deviations in this exploratory manner are ways of trying to characterise the _distribution_ of the column. A more visual way of doing so, is by directly plotting a histogram. Just like with the line plot, pandas makes this possible in one line:

In [ ]:
df['Danceability'].plot.hist()

We now have a full statistical description of the `Danceability` column, and "understand" the variable better. We can notice that most popular songs in 2019 were very danceable, with a small group of not so danceable songs. (Can you figure out which ones?)

Now, please don't be mad, but I've been keeping a trick from you this whole time. It's possible to get a statistical summary of every column in a `DataFrame` by using the [`.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) method:

In [ ]:
df.describe()

So now we know how to run `.min()`, `.max()`, `.mean()`, and `.std()` methods on `DataFrame` columns, but we also know that there is no need to do so if we've already inspected the `.describe()` output! This is often the first exploration step for a new dataset.

🧠 Can you explain what the $25\%$, $50\%$, and $75\%$ rows represent in the statistical summary?

Despite having this great statistical summary, one aspect of the dataset remains unclear. You might have noticed that columns with non-numerical `dtypes` were not included in the `.describe()` summary (e.g `Track.Name`). 

We can include them by using this argument:

In [ ]:
df.describe(include='all')

This is slightly messier, since we can't calculate the mean of `string`s or the frequency of `float`s. We can now see information about the string columns however. Dance pop is sooooo 2019... 💁‍♂️

Not all the summary statistics you might want are in the `.describe()` table. For example, the `top` value for the `Track.Name` column is fairly useless... Its `freq` is 1 meaning that they are all unique track names. That makes sense, it would be quite a coincidence if two tracks in the top 50 songs of 2019 shared a name! Nevertheless, this means that we still don't have many insights into the "typical" track name in 2019. 

Even if it's slightly more complex, we can find the 10 most common words used in the track titles:

In [ ]:
pd.Series(' '.join(df['Track.Name']).lower().split()).value_counts()[:10]

Looks like a lot of songs were featuring Justin Bieber! 🙉

🧠🧠 Can you tell what's happening here? Take the time to go through each step. If you haven't seen them before, read about the [`.join()`](https://www.w3schools.com/python/ref_string_join.asp) and [`.split()`](https://www.w3schools.com/python/ref_string_split.asp) string methods.

You might have noticed that `.describe()` returns a `DataFrame`... which means that we can manipulate and analyse it like any other pandas object!

💪 Can you figure out which column has the highest mean? You can of course manually inspect the `.describe()` table, but you should write a function called `.highest_mean()` which takes the `describe_df`, and returns the _column name_ of the column with the highest mean value. You should be able to fit your code in one line!

In [ ]:
def highest_mean(describe_df):
    # INSERT YOUR CODE HERE

def test_highest_mean():
    assert highest_mean(df.describe()) == 'Length.'
    print('Success! 🎉')

test_highest_mean()

## 2. Group-By

Calculating summary statistics for our dataset has given us a better understanding of the variables. However in data science, we often look to answer specific questions that provide _business value_. These questions are rarely as simple as "what's the average bpm of the most popular songs of 2019?". They often involve _combining_ statistics and aggregate measures of _several_ columns. For example, "what's the most danceable genre?". This kind of questions can bring real actionable insights to businesses, therefore knowing how to answer them is important!

To calculate statistics over several columns, we need to understand the concept of _group-by_. According to the excellent pandas [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html), a group-by operation has three steps:
> - Splitting the data into groups based on some criteria.  
> - Applying a function to each group independently.  
> - Combining the results into a data structure.

So for the example of "the most danceable genre", we need to:
- split the dataset by `Genre`
- average the `Danceability` values to each `Genre` independently
- concatenate those averages into a new dataframe with a `Genre` labeled index

This only describes the group-by operation of course, and we'll still have to find the max `Danceability` row.

Dancing put aside, the second group-by step ("applying a function") can take different forms. During this lecture we'll focus on _aggregation_ and _filtering_ functions.

### 2.1 Aggregation

According to the documentation:

> Aggregation functions compute a summary statistic (or statistics) for each group

In other words, they are "reduce" type functions. Their input is a collection and they return a single value. Common examples of aggregation functions are the summary statistics we saw in section 1, i.e `.mean()`, `.std()`, `.max()`, ... Let's find out the most danceable genres by using an aggregation group-by on our dataset. 

In pandas, group-by starts with the [`.groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) method:

In [ ]:
df.groupby('Genre')

It returns a `DataFrameGroupBy` object that has no readable output. One way of looking into this object is using known group _values_:

In [ ]:
df.groupby('Genre').get_group('edm')

Here, we've accessed the `edm` group of the `GroupBy` object, which is equivalent to all the dataset rows with `Genre` == `edm`. This means that so far, we've only achieved the same a boolean mask selection. 

To truly leverage the hidden power of `.groupby()`, we have to "apply functions to the groups independently". This is done through methods on the `GroupBy` object. ([Here](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html)'s the complete list).

Some of these methods are aggregation methods, such as `.mean()`. It will return the mean value of every column of the original `DataFrame` _per group_:

In [ ]:
df.groupby('Genre').mean().head()

Notice how the `Genre` is now the index of the returned `DataFrame`. We can already use this to mine for patterns and trends. For instance, you might observe that `edm` is less danceable than `country rap`... 🤠

Let's build a ranking of the 5 most danceable genres. Since the object returned by the `.mean()` method on the `GroupBy` is a `DataFrame`, we can use our trusted selection and sorting methods:

In [ ]:
df_by_genre = df.groupby(by='Genre').mean()
df_by_genre['Danceability'].sort_values(ascending=False)[:5]

Congratulations to Spanish R&B for lighting up the clubs in 2019! 🔥

### 2.2 Filtration

Sad songs are canceled 🙅‍♀️! 2020 has been too much already, and in an effort to lift our spirits, we decide to ban all music genres that create negative music. This means that the summary table we obtained with `.describe()` isn't valid anymore... those negative genres shouldn't be counted in the statistics! 

Luckily our dataset has a `Valence` column, which according to its [online description](https://www.kaggle.com/leonardopena/top50spotify2019):

> The higher the value, the more positive mood for the song.

Perfect! We know from our summary statistics that the 25% percentile for this variable is at 38.25. Let's filter out genres where the mean `Valence` < 40, and that should leave us with positive mindsets only 🙃.

This is an example of a _filtration_ function in a group-py operation. Its input is a collection of values and it returns a boolean value, depending on if the _group_ should be filtered or not.

Group-by filtration in pandas uses the [`.filter()`](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#filtration) method. It takes a [`lambda function`](https://stackoverflow.com/questions/16501/what-is-a-lambda-function) as input, and returns the filtered and concatenated dataframe. The lambda will operate on each group independently. In our case, we wish our lambda to do two things:
- calculate the mean of the `Valence`
- return False if the mean < 40

We can write it all in one line as such:

In [ ]:
happy_df = df.groupby('Genre').filter(lambda df: df['Valence.'].mean() > 40.)
happy_df.head()

This looks similar to before, but if we take a closer look at the shape of the new `DataFrame`, we can notice that some rows were removed:

In [ ]:
happy_df.shape

We did it! No more sad genres to poison our souls, only shiny happy people holding hands 🙆‍♂️

💪 Can you figure out which genres we have banned? You could try to find the differences between `df` and `happy_df`, or use group-by with aggregation function `.mean()` to find the genres with `Valence.` < 40.

💪💪 A radio station is updating their strategy, and need to know the total airtime of each genre. Let's assume that our dataset is the complete list of songs they have played recently. In the cell below, write a function that takes our dataset, and returns a `Series` with `Genre` as index, and the summed `Length.` of each genre's songs as data.

In [ ]:
def analyse_airtime(df):
    # INSERT CODE HERE

def test_analyse_airtime():
    airtime_df = analyse_airtime(df)
    assert airtime_df.loc['edm'] == 656
    print('Success! 🎉')
    print('The top 5 genres with the most airtime are:\n')
    print(airtime_df.sort_values(ascending=False)[:5])

test_analyse_airtime()

## 3. Correlation

A common question asked about data is how some columns relate to others. Correlation is a statistical measure of how changes in one variable coincide with changes in another. This can be useful for prediction, modeling, inference, or just understanding more about the processes described by the data. Realpython has a great in-depth tutorial [here](https://realpython.com/numpy-scipy-pandas-correlation-python/) which explains correlation and how to measure it in python.

Let's imagine we are trying to understand more about music through our dataset. We are trying to get a "feel" of the mechanisms behind our variables. We hypothesize that the biggest factor affecting danceability is a steady, snappy, fast beat. 🥁 We also believe that fast beats are a good indicator of a song with high energy. Therefore, we guess that `Energy` is _positively correlated_ with `Danceability`. But these are all suppositions. We'd like a quick sanity check on our "theory of music" by exploring the data.

One way of assessing correlation is to plot the variables in question:



In [ ]:
df[['Energy', 'Danceability']].plot.line()

Highly positively correlated lines are expected to vary in the same direction at the same time. It is not so clear here, so we can view a scatter plot instead:

In [ ]:
df.plot.scatter(x='Energy', y='Danceability')

By plotting the `Danceability` vs the `Energy`, we can see that there is no apparent correlation between the variables. 

This leads us to rethink our theory of `Energy`. Perhaps high energy is less about the rhythmic section, and more about the tonality and the harmonics of the song. In this case, we expect `Energy` to be positively correlated with `Valence.` which describes the "positivity" of the tune.

Let's take a look at the scatter plot:

In [ ]:
df.plot.scatter(x='Energy', y='Valence.')

The correlation is not strong, but we can notice a general trend: when `Energy` is high, `Valence.` is more likely to be high too. pandas also offers methods to directly measure the correlation between variables:

In [ ]:
df['Energy'].corr(df['Valence.'])

Here, we measured the pearson r between `Energy` and `Valence.`, showing indeed that the two are positively correlated. 

⚠️ Analysing correlations of variable is an entire field of statistics requiring formal tests and techniques. This was a simple example of how pandas makes it easy to explore various aspects of a dataset, including relationships between variables, and is in no way a replacement for sound statistical analysis.

## 4. Calculating New Variables

We've covered several ways of getting insights from the dataset. But wouldn't it be cool if we could _add_ insights to the dataset? 😎

We'll often need to save new variables. This could be because we're adding useful statistics, or calculating new values entirely. 

### 4.1 Normalisation

Normalisation is common in machine learning because of a process called [feature scaling](https://en.wikipedia.org/wiki/Feature_scaling). In short, some models train better if variables have means ~ 0 and standard deviations ~ 1. Let's normalise our `Energy` column!

First we need to define a normalisation function. In this case, we'll use [z-score normalisation](https://en.wikipedia.org/wiki/Feature_scaling#Standardization_(Z-score_Normalization)).

In [ ]:
def normalise(x, mean, std):
    return (x - mean) / std

To apply this function on our `Energy`, we need to know the mean and the standard deviation of the column.

In [ ]:
mean = df['Energy'].mean()
std = df['Energy'].std()

Now, we can wrap our function, `mean` value, and `std` value in a `lambda`. Then we can use this `lambda` in the `.apply()` method, which will calculate the normalised `Energy` values for each row. Finally, we can assign the returned `Series` to a new column name:

In [ ]:
df['Energy.Norm'] = df['Energy'].apply(lambda x: normalise(x, mean, std))
df['Energy.Norm'].head()

Let's check that the new column is indeed normalised:

In [ ]:
print('This is the normalised mean:')
print(df['Energy.Norm'].mean())
print('This is the normalised standard deviation:')
print(df['Energy.Norm'].std())

If we ignore the floating precision, those values look good! 🎊 We can also directly check the effect of normalisation on the distribution of the variable:

In [ ]:
df['Energy'].plot.hist()

In [ ]:
df['Energy.Norm'].plot.hist()

As expected, the distribution _shape_ remains unchanged, but its x-axis range is shifted to be centered around 0 and standard deviation = 1. And that should make our ML models happy 😃

### 4.2 Calculating a New Variable

📢 The data scientists at spotify have made a new discovery!  
How lit 🔥 a song is can be calculated using the following equation:

$$
🔥 = 0.1337 \times \frac{(danceability + popularity)}{(- loudness)}
$$

We would like to add this to our dataset so that we can display the "litness" of a song to users.

💪💪 Create a new column called `Litness` in the dataset. To achieve this, complete the `.litness()` function below. Then use `.apply()` to apply the function to our dataset. Pro-tip: Think about the input to your `lambda` function. Extra-tip: Since `litness` is a function that requires data from several columns, we want to apply it per row. i.e You'll need to use `axis=1` here... 

💪 What's the most lit song of 2019?

In [ ]:
import math

def litness(danceability, popularity, loudness):
    # INSERT YOUR CODE HERE

def add_litness(df):
    # INSERT YOUR CODE HERE
    return df

def test_litness():
    lit_df = add_litness(df)
    assert lit_df['Litness'].idxmax() == 13
    assert math.isclose(lit_df.loc[13, 'Litness'], 10.696)
    print('Success! 🎉')
    print('\nThe most lit song of 2019 is:')
    print(lit_df.loc[13])

test_litness()

## 5. Summary

Today, we carried out a comprehensive data exploration of **tabular data**. First, we calculated **summary statistics** to get an overview of the values in our pandas `DataFrame`. We also learned the **group-by** process, and used it to **answer questions** about the dataset. Then, we probed how several variables **correlated** to one another. Finally, we **normalised** a column, and **calculated a new variable** using a mathematical model. Overall, we got a pretty good idea of what music was popular in 2019. 🔥

This concludes the data exploration section about tabular data. You should have all the tools necessary to manipulate tables in python using pandas. Next, we will focus on other data types, such as time series, text, images, and geospatial data.


# Resources

## Core Resources
- [Exploratory data analysis notebook](https://nbviewer.jupyter.org/github/Tanu-N-Prabhu/Python/blob/master/Exploratory_data_Analysis.ipynb)  
A great example of data exploration carried out using pandas
- [Python data science handbook](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html)  
The python data science handbook introduction to pandas.

## Additional Resources

- [auqntecon pandas course](https://python-programming.quantecon.org/pandas.html)
Jupyter notebook with basic financial analysis using pandas
- [Pandas tutorial](https://bitbucket.org/hrojas/learn-pandas/src/master/)
- [Exploratory data analysis with pandas](https://www.kaggle.com/kashnitsky/topic-1-exploratory-data-analysis-with-pandas)  
In-depth Kaggle kernel of example data exploration with pandas
